In [14]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [15]:
felv = pd.read_csv("Desktop/Pazmany_Ossz.csv")

In [16]:
felv.Kar.unique()

array(['PPKE.BTK', 'PPKE.ITK', 'PPKE.JAK', 'PPKE.VJK'], dtype=object)

In [58]:
from bokeh.plotting import figure, output_file, output_notebook,  show, gridplot
from bokeh.models import ColumnDataSource


In [44]:
output_notebook()

BokehJS successfully loaded.

In [103]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

In [104]:
allamiak = felv[(felv.Tipus == "A")]
btk = ColumnDataSource(allamiak[(allamiak.Kar == "PPKE.BTK")])
itk = ColumnDataSource(allamiak[(allamiak.Kar == "PPKE.ITK")])
jak = ColumnDataSource(allamiak[(allamiak.Kar == "PPKE.JAK")])
vjk = ColumnDataSource(allamiak[(allamiak.Kar == "PPKE.VJK")])
btk

In [105]:
p = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=450, plot_height=450)
p.scatter("Ev", "Ossz_Jel",  source = btk, legend ="BTK", color = "red")
p.line("Ev", "Ossz_Jel",  source = btk, legend ="BTK", color = "red")

In [106]:
p2 = figure(title="Felvettek szama osszesen", tools=TOOLS, plot_width=450, plot_height=450)
p2.scatter("Ev", "Ossz_Felv",  source = btk, legend ="BTK", color = "red")
p2.line("Ev", "Ossz_Felv",  source = btk, legend ="BTK", color = "red")

In [95]:
s = gridplot([[p,p2]])
show(s)

ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 13a9b765-648f-4f1b-889c-5c3347c16f81]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 565cfb23-dc15-437f-a171-952962dc5303]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 6df38879-6748-4544-87cb-651394f0105f]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 463fd605-1e51-4ef4-a858-1c1b5042c20a
ERRO

In [107]:
p.scatter("Ev", "Ossz_Jel",  source = itk, legend ="ITK", color = "orange")
p.line("Ev", "Ossz_Jel",  source = itk, legend ="ITK", color = "orange")
p2.scatter("Ev", "Ossz_Felv",  source = itk, legend ="ITK", color = "orange")
p2.line("Ev", "Ossz_Felv",  source = itk, legend ="ITK", color = "orange")

p.scatter("Ev", "Ossz_Jel",  source = vjk, legend ="VJK", color = "green")
p.line("Ev", "Ossz_Jel",  source = vjk, legend ="VJK", color = "green")
p2.scatter("Ev", "Ossz_Felv",  source = vjk, legend ="VJK", color = "green")
p2.line("Ev", "Ossz_Felv",  source = vjk, legend ="VJK", color = "green")

p.scatter("Ev", "Ossz_Jel",  source = jak, legend ="JAK", color = "blue")
p.line("Ev", "Ossz_Jel",  source = jak, legend ="JAK", color = "blue")
p2.scatter("Ev", "Ossz_Felv",  source = jak, legend ="JAK", color = "blue")
p2.line("Ev", "Ossz_Felv",  source = jak, legend ="JAK", color = "blue")

In [108]:
s = gridplot([[p,p2]])
show(s)

ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 13a9b765-648f-4f1b-889c-5c3347c16f81]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 565cfb23-dc15-437f-a171-952962dc5303]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 6df38879-6748-4544-87cb-651394f0105f]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 463fd605-1e51-4ef4-a858-1c1b5042c20a
ERRO

In [109]:
p = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=450, plot_height=450)
p.scatter("Ev", "Elso_Jel",  source = btk, legend ="BTK", color = "red")
p.line("Ev", "Elso_Jel",  source = btk, legend ="BTK", color = "red")
p.scatter("Ev", "Elso_Jel",  source = itk, legend ="ITK", color = "orange")
p.line("Ev", "Elso_Jel",  source = itk, legend ="ITK", color = "orange")
p.scatter("Ev", "Elso_Jel",  source = vjk, legend ="VJK", color = "green")
p.line("Ev", "Elso_Jel",  source = vjk, legend ="VJK", color = "green")
p.scatter("Ev", "Elso_Jel",  source = jak, legend ="JAK", color = "blue")
p.line("Ev", "Elso_Jel",  source = jak, legend ="JAK", color = "blue")
s = gridplot([[p,p2]])
show(s)

ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 13a9b765-648f-4f1b-889c-5c3347c16f81]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 565cfb23-dc15-437f-a171-952962dc5303]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 6df38879-6748-4544-87cb-651394f0105f]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 463fd605-1e51-4ef4-a858-1c1b5042c20a
ERRO

In [113]:
p = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=350, plot_height=350)
p.scatter( "Elso_Jel",  "Ossz_Felv", source = btk, legend ="BTK", color = "red")
p.line("Elso_Jel",  "Ossz_Felv",  source = btk, legend ="BTK", color = "red")
p2 = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=350, plot_height=350)
p2.scatter("Elso_Jel",  "Ossz_Felv",  source = itk, legend ="ITK", color = "orange")
p2.line("Elso_Jel",  "Ossz_Felv",  source = itk, legend ="ITK", color = "orange")
p3 = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=350, plot_height=350)
p3.scatter("Elso_Jel",  "Ossz_Felv",  source = vjk, legend ="VJK", color = "green")
p3.line("Elso_Jel",  "Ossz_Felv",  source = vjk, legend ="VJK", color = "green")
p4 = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=350, plot_height=350)
p4.scatter("Elso_Jel",  "Ossz_Felv",  source = jak, legend ="JAK", color = "blue")
p4.line("Elso_Jel",  "Ossz_Felv",  source = jak, legend ="JAK", color = "blue")
s = gridplot([[p,p2], [p3, p4]])
show(s)

ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 13a9b765-648f-4f1b-889c-5c3347c16f81]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 565cfb23-dc15-437f-a171-952962dc5303]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 6df38879-6748-4544-87cb-651394f0105f]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 463fd605-1e51-4ef4-a858-1c1b5042c20a
ERRO

In [246]:
## correlation 
#np.corrcoef(allamiak["Ossz_Jel", "Elso_Jel"])
cor = np.corrcoef(allamiak[["Ossz_Jel", "Elso_Jel", "Nappali_Jel", "Allami_Jel", "Ossz_Felv", "Nappali_Felv", "Allami_Felv"]])
#allamiak
#cor.size
cor = np.corrcoef(allamiak[["Ossz_Jel", "Elso_Jel", "Nappali_Jel", "Allami_Jel", "Ossz_Felv", "Nappali_Felv", "Allami_Felv"]].as_matrix().T)
cor
x = ["Ossz_Jel", "Elso_Jel", "Nappali_Jel", "Allami_Jel", "Ossz_Felv", "Nappali_Felv", "Allami_Felv"]
trans = np.transpose([np.tile(x, len(x)), np.repeat(x, len(x))])

tmp_df = pd.DataFrame(data = {"cor" : cor.flatten(), "A": trans.T.flatten()[0:49], "B": trans.T.flatten()[49:197]})
tmp_df

,A,B,cor
0,Ossz_Jel,Ossz_Jel,1.000000
1,Elso_Jel,Ossz_Jel,0.924913
2,Nappali_Jel,Ossz_Jel,0.958203
3,Allami_Jel,Ossz_Jel,0.978995
4,Ossz_Felv,Ossz_Jel,0.961071
5,Nappali_Felv,Ossz_Jel,0.886703
6,Allami_Felv,Ossz_Jel,0.811004
7,Ossz_Jel,Elso_Jel,0.924913
8,Elso_Jel,Elso_Jel,1.000000
9,Nappali_Jel,Elso_Jel,0.838897


In [259]:
corgram = ColumnDataSource(tmp_df)

TOOLS = "resize,hover,save,pan,box_zoom,wheel_zoom"
A = tmp_df.A
B = tmp_df.B
A.values.T.tolist()

['Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv']

In [265]:

p = figure(title="Szuper corgram",
    x_range=x, y_range=x,
    x_axis_location="above", plot_width=500, plot_height=500,
    toolbar_location="left", tools=TOOLS)

colors = [
    "#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce",
    "#ddb7b1", "#cc7878", "#933b41", "#550b1d"
]

#values = colors[tmp_df.cor.values * 4 + 4]
c = np.ceil(tmp_df.cor.values * 4 + 4)
#values = [colors[int(i)] for i in c]
#colors[int(8.0)]
values = [colors[int(i)] for i in c]
values
corgram.add(data = values, name = "colors")
corgram.add(data = np.tile([1, 2, 3, 4, 5, 6, 7], len(x)), name = "x_range")
corgram.add(data = np.repeat([1, 2, 3, 4, 5, 6, 7], len(x)), name = "y_range")
#corgram["colors"] = values
p.rect(A, B, 1, 1, line_color=None)
#show(p)

p.rect("x_range",
          "y_range",
          1, 1,
          source = corgram,
          color="colors")

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

show(p)  



ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 13a9b765-648f-4f1b-889c-5c3347c16f81]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 565cfb23-dc15-437f-a171-952962dc5303]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 6df38879-6748-4544-87cb-651394f0105f]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 463fd605-1e51-4ef4-a858-1c1b5042c20a
ERRO

ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 13a9b765-648f-4f1b-889c-5c3347c16f81]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 565cfb23-dc15-437f-a171-952962dc5303]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 6df38879-6748-4544-87cb-651394f0105f]


ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 13a9b765-648f-4f1b-889c-5c3347c16f81]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 565cfb23-dc15-437f-a171-952962dc5303]
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gree [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 6df38879-6748-4544-87cb-651394f0105f]


ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 1f66f0c0-1801-4038-b1bb-5a1949303ca1
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 9ff33583-0eda-4848-83de-97f75ff88a9f
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: f241e5d3-5146-4e7b-9e7e-411e288eeadc
ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 729be8e8-4e16-4664-adc3-2bb3f004c449


ERROR:C:\Program Files\Anaconda\Anaconda\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 1f66f0c0-1801-4038-b1bb-5a1949303ca1
